In [6]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import cv2
import torch
import yaml
import shutil
import json
import random
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from glob import glob
from ultralytics import YOLO
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from pathlib import Path

In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICDES"] = "0"

In [7]:
# 원본 데이터셋 경로 설정 - 학습/검증 데이터가 이미 분리되어 있습니다.
# 실제 경로로 반드시 변경해주세요!
TRAIN_IMAGES_SOURCE_DIR = Path('C:/ssafy/AIoT-AI/Warehouse_Safety_Data/Training/images')
TRAIN_LABELS_SOURCE_DIR = Path('C:/ssafy/AIoT-AI/Warehouse_Safety_Data/Training/labels')
VAL_IMAGES_SOURCE_DIR = Path('C:/ssafy/AIoT-AI/Warehouse_Safety_Data/Validation/images')
VAL_LABELS_SOURCE_DIR = Path('C:/ssafy/AIoT-AI/Warehouse_Safety_Data/Validation/labels')

# 전처리된 YOLO 데이터셋이 저장될 최종 출력 경로
OUTPUT_BASE_DIR = 'yolo_dataset'

# 새로운 클래스 매핑을 요구사항에 맞춰 정의합니다.
# 키(key)는 원본 클래스 ID이며, 값(value)은 새롭고 통합된 영어 레이블입니다.
# class_mapping에 키로 존재하지 않는 원본 클래스 ID는 모두 데이터셋에서 제외됩니다.
class_mapping = {
    "WO-01": "WORKER",                # 작업자(작업복 착용)
    "WO-02": "WORKER",                # 작업자(작업복 미 착용)
    "WO-03": "HANDLING_EQUIPMENT",    # 화물트럭 -> 화물운반기기
    "WO-04": "HANDLING_EQUIPMENT",    # 지게차 -> 화물운반기기
    "WO-05": "HANDLING_EQUIPMENT",    # 핸드파레트카 -> 화물운반기기
    "WO-06": "HANDLING_EQUIPMENT",    # 롤테이너 -> 화물운반기기
    "WO-07": "HANDLING_EQUIPMENT",    # 운반수레 -> 화물운반기기
    "WO-08": "SMOKING_ACTIVITY",      # 흡연
    "SO-02": "STACKED_MATERIAL",      # 적재물류(그룹)
    "SO-03": "INDIVIDUAL_MATERIAL",   # 물류(개별)
    "SO-21": "FOREIGN_OBJECT_WET",    # 이물질(물,기름)
    "SO-22": "FLAMMABLE_MATERIAL",    # 가연물,인화물(목재,섬유,석유통)
    "UA-06": "MATERIAL_COLLAPSE",     # 물류 붕괴
    "UA-13": "MATERIAL_COLLAPSE",     # 물류 붕괴
    "UA-07": "IMPROPER_GEAR",         # 복장(안전모, 안전화)미착용
    "UA-20": "SMOKING_VIOLATION"      # 비 흡연 구역 내 흡연
}

# class_mapping의 값들을 기반으로 최종 클래스 목록을 생성합니다.
# 이를 통해 최종 목표 클래스만 포함되도록 합니다.
all_classes = sorted(list(set(class_mapping.values())))
class_to_id = {name: i for i, name in enumerate(all_classes)}
id_to_class = {i: name for i, name in enumerate(all_classes)}

print(f"최종 클래스 개수: {len(all_classes)}")
print(f"최종 정의된 클래스: {all_classes}")
print(f"최종 클래스 ID 매핑: {class_to_id}")

최종 클래스 개수: 10
최종 정의된 클래스: ['FLAMMABLE_MATERIAL', 'FOREIGN_OBJECT_WET', 'HANDLING_EQUIPMENT', 'IMPROPER_GEAR', 'INDIVIDUAL_MATERIAL', 'MATERIAL_COLLAPSE', 'SMOKING_ACTIVITY', 'SMOKING_VIOLATION', 'STACKED_MATERIAL', 'WORKER']
최종 클래스 ID 매핑: {'FLAMMABLE_MATERIAL': 0, 'FOREIGN_OBJECT_WET': 1, 'HANDLING_EQUIPMENT': 2, 'IMPROPER_GEAR': 3, 'INDIVIDUAL_MATERIAL': 4, 'MATERIAL_COLLAPSE': 5, 'SMOKING_ACTIVITY': 6, 'SMOKING_VIOLATION': 7, 'STACKED_MATERIAL': 8, 'WORKER': 9}


In [9]:
def convert_json_to_yolo(json_file_path, output_dir, image_width, image_height, class_mapping, class_to_id):
    """
    단일 JSON 레이블 파일을 YOLO 형식(.txt)으로 변환합니다.
    클래스 재매핑 및 제외 로직이 적용됩니다.
    """
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    image_id = Path(json_file_path).stem # 파일명에서 확장자 제거

    yolo_labels = []
    
    if "Learning data info." in data and "annotation" in data["Learning data info."]:
        for annotation in data["Learning data info."]["annotation"]:
            original_class_name = annotation["class_id"]
            
            # 클래스 매핑 적용
            mapped_class_name = class_mapping.get(original_class_name)

            # 새로운 매핑에 정의되지 않은 (즉, 제외된) 클래스인 경우 건너뛰기
            if mapped_class_name is None:
                continue

            # 매핑된 클래스 이름이 최종 class_to_id 목록에 있는지 확인
            if mapped_class_name not in class_to_id:
                print(f"경고: 매핑된 클래스 '{mapped_class_name}' (원본 '{original_class_name}'에서)가 최종 정의된 클래스에서 발견되지 않았습니다. {json_file_path}의 어노테이션을 건너뜁니다.")
                continue

            class_id = class_to_id[mapped_class_name]
            coords = annotation["coord"]
            
            # 'box' 타입 어노테이션 처리
            if annotation["type"] == "box":
                if len(coords) == 4:
                    x_min, y_min, width, height = coords
                    
                    # YOLO 형식에 맞는 정규화된 center_x, center_y, width, height 계산
                    center_x = (x_min + width / 2) / image_width
                    center_y = (y_min + height / 2) / image_height
                    normalized_width = width / image_width
                    normalized_height = height / image_height

                    yolo_labels.append(f"{class_id} {center_x:.6f} {center_y:.6f} {normalized_width:.6f} {normalized_height:.6f}")
                else:
                    print(f"경고: {json_file_path}의 박스 어노테이션에 예상치 못한 좌표 개수가 있습니다: {len(coords)}. 건너뜁니다.")
            # 폴리곤 어노테이션은 현재 건너뜁니다.
            elif annotation["type"] == "polygon":
                print(f"경고: 클래스 '{original_class_name}'에 대한 폴리곤 어노테이션이 발견되었습니다. 현재는 박스 어노테이션만 처리합니다. 건너뜁니다.")
                continue

    output_txt_path = Path(output_dir) / f"{image_id}.txt"
    # 쓸 레이블이 있는 경우에만 .txt 파일을 생성합니다.
    if yolo_labels:
        with open(output_txt_path, 'w', encoding='utf-8') as f:
            for line in yolo_labels:
                f.write(line + '\n')
        return True
    else:
        # 이 경우, 유효한 레이블이 없어 해당 이미지와 레이블을 최종 데이터셋에 포함하지 않습니다.
        return False


def process_and_copy_split_dataset(source_images_dir, source_labels_dir, target_images_dir, target_labels_dir, class_mapping, class_to_id):
    """
    분할된 원본 데이터셋(이미지 및 JSON 레이블)을 YOLO 형식으로 변환하여 대상 디렉토리로 복사합니다.
    """
    Path(target_images_dir).mkdir(parents=True, exist_ok=True)
    Path(target_labels_dir).mkdir(parents=True, exist_ok=True)

    json_files = list(source_labels_dir.glob('*.json'))
    
    processed_count = 0
    skipped_count = 0

    print(f"'{source_labels_dir}'에서 JSON 파일 처리 중...")
    for json_file in tqdm(json_files):
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                json_data = json.load(f)
            
            if "Source data Info." in json_data and "source_data_ID" in json_data["Source data Info."]:
                image_id = json_data["Source data Info."]["source_data_ID"]
                image_extension = json_data["Source data Info."].get("file_extension", "jpg")
                image_filename = f"{image_id}.{image_extension}"
                image_path = source_images_dir / image_filename

                if not image_path.exists():
                    print(f"경고: {json_file}에 대한 이미지 파일을 찾을 수 없습니다. 건너뜁니다: {image_path}")
                    skipped_count += 1
                    continue

                if "Raw data Info." in json_data and "resolution" in json_data["Raw data Info."]:
                    width, height = json_data["Raw data Info."]["resolution"]
                else:
                    try:
                        img = Image.open(image_path)
                        width, height = img.size
                    except Exception as e:
                        print(f"경고: {image_path}의 이미지 해상도를 읽는 중 오류 발생: {e}. 건너뜁니다.")
                        skipped_count += 1
                        continue

                # YOLO 형식으로 변환. 유효한 레이블이 생성된 경우에만 이미지와 레이블을 복사.
                if convert_json_to_yolo(json_file, target_labels_dir, width, height, class_mapping, class_to_id):
                    shutil.copy(image_path, target_images_dir / image_path.name)
                    processed_count += 1
                else:
                    # 유효한 레이블이 없거나, 모든 레이블이 제외되어 빈 .txt 파일이 생성되지 않은 경우
                    # 해당 이미지는 최종 데이터셋에 포함하지 않습니다.
                    skipped_count += 1
                    # print(f"정보: 매핑/제외 후 {json_file}에 유효한 레이블이 없습니다. 이미지를 건너뜁니다.")

            else:
                print(f"경고: {json_file}에서 'Source data Info.source_data_ID'를 찾을 수 없습니다. 건너뜁니다.")
                skipped_count += 1
        except json.JSONDecodeError as e:
            print(f"오류: {json_file} JSON 파일 디코딩 중 오류 발생: {e}. 건너뜁니다.")
            skipped_count += 1
        except Exception as e:
            print(f"오류: {json_file} 처리 중 예상치 못한 오류 발생: {e}. 건너뜁니다.")
            skipped_count += 1
    
    print(f"완료. 처리된 파일: {processed_count}, 건너뛴 파일: {skipped_count}")
    return processed_count


def prepare_yolo_dataset(output_base_dir, class_mapping, class_to_id):
    """
    미리 분할된 원본 데이터셋을 YOLO 학습에 적합한 디렉토리 구조로 변환하고 복사합니다.
    """
    output_images_train_dir = Path(output_base_dir) / 'images' / 'train'
    output_labels_train_dir = Path(output_base_dir) / 'labels' / 'train'
    output_images_val_dir = Path(output_base_dir) / 'images' / 'val'
    output_labels_val_dir = Path(output_base_dir) / 'labels' / 'val'

    # 기존 출력 디렉토리 정리 및 재생성
    if Path(output_base_dir).exists():
        shutil.rmtree(output_base_dir)
    
    print(f"'{output_base_dir}'에 최종 YOLO 데이터셋 디렉토리 구조 생성 중...")
    output_images_train_dir.mkdir(parents=True, exist_ok=True)
    output_labels_train_dir.mkdir(parents=True, exist_ok=True)
    output_images_val_dir.mkdir(parents=True, exist_ok=True)
    output_labels_val_dir.mkdir(parents=True, exist_ok=True)

    print("\n--- 학습 데이터 전처리 및 복사 ---")
    train_processed_count = process_and_copy_split_dataset(
        TRAIN_IMAGES_SOURCE_DIR, TRAIN_LABELS_SOURCE_DIR,
        output_images_train_dir, output_labels_train_dir,
        class_mapping, class_to_id
    )

    print("\n--- 검증 데이터 전처리 및 복사 ---")
    val_processed_count = process_and_copy_split_dataset(
        VAL_IMAGES_SOURCE_DIR, VAL_LABELS_SOURCE_DIR,
        output_images_val_dir, output_labels_val_dir,
        class_mapping, class_to_id
    )
    
    print(f"\n최종 YOLO 데이터셋 통계:")
    print(f"학습 이미지 수: {train_processed_count}")
    print(f"검증 이미지 수: {val_processed_count}")

    # dataset.yaml 파일 생성
    data_yaml = {
        'path': str(Path(output_base_dir).resolve()),
        'train': 'images/train',
        'val': 'images/val',
        'nc': len(all_classes),
        'names': all_classes
    }

    with open(Path(output_base_dir) / 'dataset.yaml', 'w') as f:
        yaml.dump(data_yaml, f, sort_keys=False)

    print(f"\n데이터셋 전처리가 완료되었습니다. '{output_base_dir}'에 YOLO 데이터셋이 생성되었습니다.")
    print(f"학습을 위한 'dataset.yaml' 파일이 생성되었습니다.")

# 데이터 전처리 실행
prepare_yolo_dataset(OUTPUT_BASE_DIR, class_mapping, class_to_id)

'yolo_dataset'에 최종 YOLO 데이터셋 디렉토리 구조 생성 중...

--- 학습 데이터 전처리 및 복사 ---
'C:\ssafy\AIoT-AI\Warehouse_Safety_Data\Training\labels'에서 JSON 파일 처리 중...


100%|██████████| 33371/33371 [04:45<00:00, 116.88it/s]


완료. 처리된 파일: 33365, 건너뛴 파일: 6

--- 검증 데이터 전처리 및 복사 ---
'C:\ssafy\AIoT-AI\Warehouse_Safety_Data\Validation\labels'에서 JSON 파일 처리 중...


100%|██████████| 4172/4172 [01:11<00:00, 58.29it/s]

완료. 처리된 파일: 4172, 건너뛴 파일: 0

최종 YOLO 데이터셋 통계:
학습 이미지 수: 33365
검증 이미지 수: 4172

데이터셋 전처리가 완료되었습니다. 'yolo_dataset'에 YOLO 데이터셋이 생성되었습니다.
학습을 위한 'dataset.yaml' 파일이 생성되었습니다.


In [10]:
def seed_everything(seed=40):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
seed_everything()

In [ ]:
# 사전 학습된 YOLOv8m 모델 로드
model = YOLO('yolov8m.pt')

# 모델 학습
# 'data': 생성된 dataset.yaml 파일 경로
# 'epochs': 학습 에포크 수 (데이터셋 크기와 원하는 성능에 따라 조절)
# 'imgsz': 모델의 입력 이미지 크기
# 'batch': 배치 크기 (GPU 메모리에 따라 조절)
# 'name': 'runs/detect/' 아래에 결과가 저장될 폴더 이름
results = model.train(data=f'{OUTPUT_BASE_DIR}/dataset.yaml', epochs=100, imgsz=640, batch='auto', name='yolov8m_warehouse_safety_finetune_split_data', save_period=5)

100%|██████████| 49.7M/49.7M [00:11<00:00, 4.66MB/s]


Ultralytics 8.3.168  Python-3.10.18 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6141MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=yolo_dataset/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8m_warehouse_safety_finetune_split_data, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, pat

100%|██████████| 755k/755k [00:00<00:00, 7.81MB/s]

Overriding model.yaml nc=80 with nc=10

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                


  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics.nn.modules.conv.Conv             [384, 576, 3, 2]              
  8                  -1  2   3985920  ultralytics.nn.modules.block.C2f             [576, 576, 2, True]           
  9                  -1  1    831168  ultralytics.nn.modules.block.SPPF            [576, 576, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None

...

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access  (ping: 0.10.0 ms, read: 1044.1340.1 MB/s, size: 936.4 KB)


val: Scanning C:\ssafy\AIoT-AI\S13P11D103\AI\yolo_dataset\labels\val... 4172 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4172/4172 [00:02<00:00, 1681.87it/s]


val: New cache created: C:\ssafy\AIoT-AI\S13P11D103\AI\yolo_dataset\labels\val.cache
Plotting labels to runs\detect\yolov8m_warehouse_safety_finetune_split_data\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\yolov8m_warehouse_safety_finetune_split_data
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      6.31G      1.092      3.936      1.137        198        640:   2%|▏         | 37/2086 [00:49<45:57,  1.35s/it] 


KeyboardInterrupt: 

In [ ]:
DATA_YAML_PATH = "data.yaml"

with open(DATA_YAML_PATH, "r", encoding="utf-8") as f:
    data_yaml = yaml.safe_load(f)

DATASET_PATH = os.path.dirname(DATA_YAML_PATH)
TRAIN_IMAGES = os.path.join(DATASET_PATH, data_yaml["train"].replace("../", ""))
VALID_IMAGES = os.path.join(DATASET_PATH, data_yaml["val"].replace("../", ""))

print(DATASET_PATH)
print(TRAIN_IMAGES)
print(VALID_IMAGES)

In [ ]:
print(torch.cuda.is_available())

In [ ]:
class ObstacleDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_paths = glob(os.path.join(image_dir, "*.jpg"))
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = cv2.imread(self.image_paths[idx])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        return self.transform(img) if self.transform else img


transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
])

train_loader = DataLoader(ObstacleDataset(TRAIN_IMAGES, transform), batch_size=16, shuffle=True)
valid_loader = DataLoader(ObstacleDataset(VALID_IMAGES, transform), batch_size=16, shuffle=False)


def show_sample_images(image_loader):
    sample_images = next(iter(image_loader))

    fig, ax = plt.subplots(2, 3, figsize=(12, 8))

    for i, img in enumerate(sample_images[:6]):
        ax[i // 3, i % 3].imshow(img.permute(1, 2, 0).numpy())
        ax[i // 3, i % 3].axis("off")

    plt.show()


print("훈련 데이터 샘플")
show_sample_images(train_loader)

In [ ]:
IMAGE_DIR = "/kaggle/input/pothole-detection-challenge/train/images"
IMAGE_PATHS = sorted(glob(os.path.join(IMAGE_DIR, "*.jpg")))

IMG_PATH = random.choice(IMAGE_PATHS)
LABEL_PATH = IMG_PATH.replace("images", "labels").replace(".jpg", ".txt")

img = cv2.imread(IMG_PATH)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
h, w, _ = img.shape

if os.path.exists(LABEL_PATH):
    with open(LABEL_PATH, "r") as f:
        for line in f.readlines():
            cls, cx, cy, bw, bh = map(float, line.strip().split())
            x1 = int((cx - bw / 2) * w)
            y1 = int((cy - bh / 2) * h)
            x2 = int((cx + bw / 2) * w)
            y2 = int((cy + bh / 2) * h)
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 5)
            cv2.putText(img, f"Class {int(cls)}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1)
else:
    print("라벨 파일이 존재하지 않습니다:", LABEL_PATH)

# 시각화
plt.figure(figsize=(6, 6))
plt.imshow(img)
plt.axis("off")
plt.title(os.path.basename(IMG_PATH))
plt.show()

In [ ]:
model = YOLO("/kaggle/working/pothole_yolov8_train/weights/best.pt")

val_results = model.val(data=DATA_YAML_PATH, split="val")

print("검증 데이터 평가 결과:")
print(f"mAP50: {val_results.box.map50:.4f}")
print(f"mAP50-95: {val_results.box.map:.4f}")
print(f"Precision: {val_results.box.mp:.4f}")
print(f"Recall: {val_results.box.mr:.4f}")

In [2]:
def train_yolov8m(data_path='data.yaml', epochs=100, img_size=640, batch_size=16, project='runs/train', name='exp'):
    """
    Trains a YOLOv8m model.

    Args:
        data_path (str): Path to the YAML file containing dataset configuration.
        epochs (int): Number of training epochs.
        img_size (int): Image size for training.
        batch_size (int): Batch size for training.
        project (str): Project name for saving results.
        name (str): Experiment name for saving results within the project.
    """
    # Load a YOLOv8m model
    # You can load a pre-trained model for fine-tuning
    model = YOLO('yolov8m.pt')  # yolov8m.pt is the pre-trained medium model

    # If you want to train from scratch (less common for yolov8m, but possible)
    # model = YOLO('yolov8m.yaml') # Create a new model from scratch configuration

    print(f"Starting YOLOv8m training with the following parameters:")
    print(f"  Data: {data_path}")
    print(f"  Epochs: {epochs}")
    print(f"  Image Size: {img_size}")
    print(f"  Batch Size: {batch_size}")
    print(f"  Project: {project}")
    print(f"  Name: {name}")

    # Train the model
    results = model.train(
        data=data_path,
        epochs=epochs,
        imgsz=img_size,
        batch=batch_size,
        project=project,
        name=name,
        # You can add more arguments here based on your needs, e.g.:
        # device='0',         # GPU device (e.g., '0' for first GPU, or 'cpu')
        # patience=50,        # Early stopping patience
        # resume=False,       # Resume training from last checkpoint
        # lr0=0.01,           # Initial learning rate
        # lrf=0.001,          # Final learning rate (lr0 * lrf)
        # optimizer='auto',   # Optimizer: 'SGD', 'Adam', 'AdamW', 'RMSProp', 'auto'
        # workers=8,          # Number of DataLoader workers
        # val=True,           # Validate during training
        # cache=False,        # Cache images for faster training
        # cos_lr=False,       # Cosine learning rate scheduler
        # exist_ok=False,     # Don't overwrite existing results directory
        # single_cls=False,   # Treat all classes as a single class
        # save_period=10,     # Save checkpoint every 'save_period' epochs
    )